### Data Processing under Conda 

* install necessary pacakges under conda env tensorflow_p36 
* to save time, use validation dataset as training set 
 

In [ ]:
%%bash 
pip install nibabel opencv-python matplotlib keras==2.3.1 
mkdir -p h5 
mkdir -p h5/detection


In [ ]:
#change to previous 5 sites to reduce execution time 
!python data_load.py

In [ ]:
!python Stroke_segment.py local train 

### Build D-Unet container 

* Dockerfile - install necessary packages and setup entrypoints 
* build_and_push - communicate with ECR (ElasticContainerRegistry)


In [ ]:
%%bash 
cd container 
./build_and_push.sh 

In [ ]:
import boto3 
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
image_uri = "{}.dkr.ecr.{}.amazonaws.com/dunet".format(account_id, "us-west-2")
image_uri

### Training locally based on the docker image 

In [ ]:
%%bash 
mkdir -p input
mkdir -p input/data
mkdir -p input/data/atlas
mv h5 input/data/atlas

In [ ]:
!nvidia-docker run -it -v $PWD:/opt/ml 938149709838.dkr.ecr.us-west-2.amazonaws.com/dunet

### Use SageMaker Training Jobs 

In [ ]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "dunet"

#### upload data to s3 

In [ ]:
!cd ./input/data/ && aws s3 cp --recursive atlas s3://{bucket}/{prefix}/atlas

#### define s3 input and output paths 

In [ ]:
atlas_h5_data = "s3://{}/{}/atlas/".format(bucket, prefix)
outpath = "s3://{}/{}/output/".format(bucket, prefix)
repositoryUri = image_uri

#### define job_name and  and hyperparameters

In [ ]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "dunet-{}".format(str(int(timestamp))) 
job_name 

#### submit training job 

In [ ]:
atlas_input = sagemaker.inputs.TrainingInput(atlas_h5_data)

In [ ]:


estimator = sagemaker.estimator.Estimator( 
                        role=role,
                        image_uri=repositoryUri,
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        sagemaker_session=sagemaker_session,
                        volume_size=100, 
                        debugger_hook_config=False
                   )


In [ ]:
estimator.fit(inputs={"atlas":atlas_input}, job_name=job_name)


/usr/local/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
Epoch 1/150
